In [123]:
import pandas as pd

# Load the Excel file
excel_file = pd.ExcelFile('UE_data.xlsx')

# List to store processed sheets
dfs = []

# Fixed header
fixed_columns = ['country', '2014', '2015', '2016', '2017', '2018', '2020', '2021', '2023', '2024']

# Loop through each sheet
for sheet_name in excel_file.sheet_names:
    if sheet_name.startswith('Sheet'): 
        # Read metadata
        metadata = pd.read_excel(excel_file, sheet_name=sheet_name, nrows=10)
        
        # Extract metadata values
        number_of_persons_employed = metadata.iloc[4, 2]
        economic_activity = metadata.iloc[5, 2]
        information_society_indicator = metadata.iloc[6, 2]
        print('information_society_indicator', information_society_indicator)
        # Read actual data
        df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=17, header=None)
        
        # Drop empty rows/columns
        df = df.dropna(how='all')
        df = df.dropna(axis=1, how='all')
        
        # Set fixed header
        df.columns = fixed_columns

        # Add metadata columns
        df['number of persons employed'] = number_of_persons_employed
        df['economic activity'] = economic_activity
        df['information society indicator'] = information_society_indicator
        
        # Add to list
        dfs.append(df)

# Concatenate all
final_df = pd.concat(dfs, ignore_index=True)

# Display result
print(final_df.head())


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


information_society_indicator Buy cloud computing services used over the internet
information_society_indicator Enterprises buying only basic CC services, at least one of: CC_PEM, CC_PSOFT, CC_PFIL, CC_PCPU
information_society_indicator Enterprises buying at most  intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM
information_society_indicator Enterprises buying sophisticated CC services, at least one of: CC_PSEC, CC_PDB, CC_PDEV
information_society_indicator Enterprises buying sophisticated or intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM, CC_PSEC, CC_PDB, CC_PDEV
information_society_indicator Buy cloud computing services used over the internet
information_society_indicator Enterprises buying only basic CC services, at least one of: CC_PEM, CC_PSOFT, CC_PFIL, CC_PCPU
information_society_indicator Enterprises buying at most  intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM
information_society_indicator Enterprises buying s

In [124]:
final_df['information society indicator'].value_counts()

information society indicator
Buy cloud computing services used over the internet                                                                                    456
Enterprises buying only basic CC services, at least one of: CC_PEM, CC_PSOFT, CC_PFIL, CC_PCPU                                         456
Enterprises buying at most  intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM                                      456
Enterprises buying sophisticated CC services, at least one of: CC_PSEC, CC_PDB, CC_PDEV                                                456
Enterprises buying sophisticated or intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM, CC_PSEC, CC_PDB, CC_PDEV    456
Name: count, dtype: int64

In [125]:
# Create a dictionary to map old long names to short names
short_names = {
    'Buy cloud computing services used over the internet': 'Buy CC',
    'Enterprises buying only basic CC services, at least one of: CC_PEM, CC_PSOFT, CC_PFIL, CC_PCPU': 'Basic CC',
    'Enterprises buying at most  intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM': 'Intermediate CC',
    'Enterprises buying sophisticated CC services, at least one of: CC_PSEC, CC_PDB, CC_PDEV': 'Sophisticated CC',
    'Enterprises buying sophisticated or intermediate CC services, at least one of: CC_PFACC, CC_PERP, CC_PCRM, CC_PSEC, CC_PDB, CC_PDEV': 'Inter/Sophisticated CC'
}

# Apply the mapping
final_df['information society indicator'] = final_df['information society indicator'].map(short_names)

# Now check again
print(final_df['information society indicator'].value_counts())

information society indicator
Buy CC                    456
Basic CC                  456
Intermediate CC           456
Sophisticated CC          456
Inter/Sophisticated CC    456
Name: count, dtype: int64


In [126]:
# Liste des colonnes d'années
year_columns = ['2014', '2015', '2016', '2017', '2018', '2020', '2021', '2023', '2024']

# Supprimer les lignes où toutes les années sont NaN
final_df = final_df.dropna(subset=year_columns, how='all')

print(final_df)

              country           2014  2015   2016   2017   2018   2020   2021  \
0             Belgium              :     :  22.68      :      :      :  51.72   
1            Bulgaria           6.02  4.15   3.96   5.13   6.88   8.75    9.4   
2             Czechia          13.17     :  14.73  19.05  23.39  26.52  39.18   
3             Denmark          33.39  34.3  40.67  48.33  55.37  66.14   62.3   
4             Germany           9.78     :  13.87      :  19.08  30.71  38.43   
...               ...            ...   ...    ...    ...    ...    ...    ...   
2274  North Macedonia              :     :      :      :      :      :   3.71   
2275          Albania              :     :      :      :      :      :      :   
2276           Serbia              :     :      :      :      :      :  19.38   
2277          Türkiye              :     :      :      :      :      :   9.24   
2279                :  not available   NaN    NaN    NaN    NaN    NaN    NaN   

       2023 2024   number o

In [127]:
print(final_df.isnull().sum())

country                           0
2014                              0
2015                             60
2016                             60
2017                             60
2018                             60
2020                             60
2021                             60
2023                             60
2024                             60
number of persons employed        0
economic activity                 0
information society indicator     0
dtype: int64


In [130]:
import numpy as np
final_df.head()
final_df.replace(':', np.nan, inplace=True)

# Maintenant on fait le pivot (melt)
df_melted = pd.melt(
    final_df,
    id_vars=['country', 'number of persons employed', 'economic activity', 'information society indicator'],
    value_vars=['2014', '2015', '2016', '2017', '2018', '2020', '2021', '2023', '2024'],
    var_name='year',
    value_name='value'
)
df_melted = df_melted.dropna(subset=['value', 'country'])

df_melted.head()

,country,number of persons employed,economic activity,information society indicator,year,value
1,Bulgaria,10 persons employed or more,Manufacturing,Buy CC,2014,6.02
2,Czechia,10 persons employed or more,Manufacturing,Buy CC,2014,13.17
3,Denmark,10 persons employed or more,Manufacturing,Buy CC,2014,33.39
4,Germany,10 persons employed or more,Manufacturing,Buy CC,2014,9.78
5,Estonia,10 persons employed or more,Manufacturing,Buy CC,2014,13.07


In [131]:
print(df_melted.isnull().sum())
print(df_melted.shape)

country                          0
number of persons employed       0
economic activity                0
information society indicator    0
year                             0
value                            0
dtype: int64
(5147, 6)


In [132]:
df_melted.to_excel('final_output.xlsx', index=False)
